In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
from math import atan2, degrees

# --- Configuration ---
content_dir = '/content'
parquet_files = [f for f in os.listdir(content_dir) if f.endswith('.parquet')]

# --- Helper functions ---
def extract_route_coordinates(geometry):
    if geometry is None or geometry.is_empty:
        return None
    return [(coord[1], coord[0]) for coord in geometry.coords]

def compute_azimuth(p1, p2):
    """Compute azimuth from p1 to p2 in degrees (0-360)."""
    lat1, lon1 = np.radians(p1)
    lat2, lon2 = np.radians(p2)
    d_lon = lon2 - lon1
    x = np.sin(d_lon) * np.cos(lat2)
    y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(d_lon)
    az = np.degrees(atan2(x, y))
    return (az + 360) % 360

def compute_turn_angles(route):
    """
    Compute signed turn angles (delta azimuth) at each intermediate point.
    Positive = right turn, Negative = left turn, Straight ~ 0.
    """
    if route is None or len(route) < 3:
        return []
    angles = []
    for i in range(len(route) - 2):
        az1 = compute_azimuth(route[i], route[i+1])
        az2 = compute_azimuth(route[i+1], route[i+2])
        delta = (az2 - az1 + 180) % 360 - 180  # signed angle (-180 to 180)
        angles.append(delta)
    return angles

# --- Process each parquet file ---
for parquet_file in parquet_files:
    print(f"\nProcessing: {parquet_file}")
    file_path = os.path.join(content_dir, parquet_file)
    gdf = gpd.read_parquet(file_path)

    all_turn_angles = []

    for geom in gdf.geometry:
        coords = extract_route_coordinates(geom)
        if coords is None or len(coords) < 3:
            continue
        angles = compute_turn_angles(coords)
        all_turn_angles.extend(angles)

    if not all_turn_angles:
        print(f"No valid turn angles in {parquet_file}")
        continue

    # --- Plot histogram of signed turn angles ---
    plt.figure(figsize=(12, 6))
    bins = np.arange(-180, 190, 10)  # -180 to +180 in 10-degree bins
    plt.hist(all_turn_angles, bins=bins, color="#FFA500", edgecolor="black", alpha=0.8)  # Orange
    plt.xlabel("Turn Angle", fontsize=12)
    plt.ylabel("Frequency", fontsize=12)
    # plt.title(f"Distribution of Turn Angles (Signed) - {parquet_file}", fontsize=14)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"/content/signed_turn_angle_distribution_{parquet_file[:-8]}.png", dpi=150)
    plt.show()
    plt.close()
